In [ ]:
utils_path = "../utils"
models_path = "../models"
data_path = "../data/celebs_20"

import sys
import os
sys.path.insert(1, utils_path)
sys.path.insert(2, models_path)

import torch
import torch.nn as nn

import matplotlib.pyplot as plt

import numpy as np

import neptune.new as neptune
from neptune.new.types import File

from config import NEPTUNE_TOKEN

import time 
import copy
import random
from tqdm.notebook import trange, tqdm

import torchvision

from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision import datasets, transforms
import torchvision.transforms as transformss

from MBNV3 import MobileNetV3
from MBNV3CBAM import MobileNetV3CBAM

from dataloader import mixedSets
from functions import train, evaluate, confusion, unzip_files, unzip

In [ ]:
#Initialising GereralTorch class

#Report multiple hyperparameters using a dictionary
hyper_params = {
    'learning_rate': 0.001,
    'epochs': 1500,
    'batch_size': 16,
    'image_size': 112,
    'image_channels': 3,
    'output_size': len(os.listdir(data_path)) - 1,
    'num_layers': 'na',
    'train_val_test_split': [0.8, 0.1, 0.1],
    'device': 'mps',
    'model_name': 'MobileNet3 with CBAM inserted instead of SE starting from a fresh model with no preloaded weights.',
    'criterion': 'CrossEntropyLoss',
    'optimizer': 'Adam',
    'dataset': 'Celebrities 20',
    'best_model_path': 'output/MN3LCB_celebs20.pt',
    'loaded_model_path': 'output/MN3LCB_celebs20.pt',
    'save_at_end': True,
}

print(f'Output Size: {hyper_params["output_size"]}')

#Setting the device
device = torch.device(hyper_params['device'])

#checking if the model is to be a loaded one and if so loading it
if hyper_params['loaded_model_path']:
    state_dict = torch.load(hyper_params['loaded_model_path']) #Loading the state dict
    model = MobileNetV3CBAM(mode='large') #Loading the model as a backbone
    model.load_state_dict(state_dict, strict= True) #Loading the state dict into the model
    print('Model Loaded')
else:
    # Loading a fresh model
    model = MobileNetV3CBAM(mode='large')

#passing the model to the device
model.to(device)

# Setting the loss function and optimizer
criterion = nn.CrossEntropyLoss().to(device) #Setting the loss function
optimizer = torch.optim.Adam(model.parameters(), lr=hyper_params['learning_rate']) #Setting the optimizer



In [ ]:
#Setting the experiment with the API key stored in config.py
run = neptune.init(project="leothesouthafrican/Thesis", api_token=NEPTUNE_TOKEN,
                                    source_files=["*.ipynb"])

#Initialising the model
run["model"] = model
neptune_model = neptune.init_model_version(
    with_id="THES-MBNV3CBAM-10",
    project="leothesouthafrican/Thesis",
    api_token=NEPTUNE_TOKEN,
)

#Logging the hyperparameters to the run on neptune
run["hyper-parameters"] = hyper_params
run["train/images"].track_files(data_path)

#Logging the hyperparameters to the run on neptune
run["hyper-parameters"] = hyper_params
run["train/images"].track_files(data_path)



In [ ]:
#defining transforms

train_transform = transforms.Compose([
                transforms.Resize((hyper_params['image_size'],hyper_params['image_size'])),
                transforms.ToTensor(),
                transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225]),
                transforms.RandomApply([transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4),
                transforms.RandomCrop(hyper_params['image_size'], padding=2)], p=0.4),
                transforms.RandomAffine(degrees=10, translate=(0.1,0.1), scale=(0.8,1.2)),
                transforms.RandomRotation(35),
                transforms.RandomHorizontalFlip(0.2),
                transforms.RandomVerticalFlip(0.15),
                transforms.RandomErasing(0.2),
            ])

test_transform = transforms.Compose([
                transforms.Resize((hyper_params['image_size'],hyper_params['image_size'])),
                transforms.ToTensor(),
                transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])
            ])

#Logging the transforms to neptune
run["train/transforms"] = train_transform
run["test/transforms"] = test_transform

In [ ]:
# Creating the dataset
train_dataset = mixedSets(data_path, train_transform, hyper_params['train_val_test_split']).get_train_dataset()
val_dataset = mixedSets(data_path, test_transform, hyper_params['train_val_test_split']).get_val_dataset()
test_dataset = mixedSets(data_path, test_transform, hyper_params['train_val_test_split']).get_test_dataset()

# Creating the dataloaders
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                            batch_size=hyper_params['batch_size'],
                                            shuffle=True,
                                            num_workers=1)

val_loader = torch.utils.data.DataLoader(dataset=val_dataset,
                                            batch_size=hyper_params['batch_size'],
                                            shuffle=True,
                                            num_workers=1)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                            batch_size=hyper_params['batch_size'],
                                            shuffle=True,
                                            num_workers=1)

#Logging the dataset to neptune
run["train/dataset"] = train_dataset
run["val/dataset"] = val_dataset
run["test/dataset"] = test_dataset


In [ ]:
#display random images from the dataset
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    #drop axis labels
    plt.axis('off')
    plt.show()

# get some random training images
dataiter = iter(train_loader)
images, labels = next(dataiter)

# show images
imshow(torchvision.utils.make_grid(images))

#save image to output folder
torchvision.utils.save_image(torchvision.utils.make_grid(images), 'output/train_images.png')

#Logging the images to neptune
#run["train/images"].upload('output/train_images.png')

In [ ]:
train(model, criterion, optimizer, hyper_params, train_loader, val_loader, run)

In [ ]:
model.load_state_dict(torch.load(hyper_params['best_model_path'])) #Loading the state dict into the model
test_loss, test_acc = evaluate(model, test_loader, criterion, device, run) 
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

In [ ]:
neptune_model["model"].upload(hyper_params['best_model_path'])
neptune_model["validation/acc"] = test_acc
neptune_model["validation/loss"] = test_loss
neptune_model["validation/dataset"] = test_dataset

In [ ]:
confusion(model, test_loader, run, device)